In [16]:
excercise_dict={
    "lunges":
    {
        "knee":
        {
            "status": "more",
             "angle":45
        },
        "backbone":
        {
            "status": "less",
             "angle1":150
            "angle2":180
        }
       
    },
    
    "bicepcurl":
    {
         "neck":
        {
            "status": "less",
             "angle":70
        },
        "backbone":
        {
            "status": "less",
             "angle":150
        }
   
    }
    
}

In [2]:
excercise_dict["lunges"]["knee"]["angle"]

45

In [20]:
def keygenerator(data):
    angle=data[0]
    joint=data[1]
    excercise=data[2]
    status=excercise_dict[excercise][joint]["status"]
    if excercise_dict[excercise][joint]["status"]=="more":
        if angle>excercise_dict[excercise][joint]["angle1"] and angle<excercise_dict[excercise][joint]["angle2"] :
            keywords="for "+excercise+" angle "+status+" for "+joint
        else:
            keywords=" good work"


    elif excercise_dict[excercise][joint]["status"]=="less":
        if angle<excercise_dict[excercise][joint]["angle"]:
            keywords="for "+excercise+" angle "+status+" for "+joint
        else:
            keywords=" good work continue excercise"
    return keywords
    

# Loading

In [4]:
import autocompleter 
autocompl = autocompleter.Autocompleter()

In [5]:
df = autocompl.import_json("sample.json")
df.shape, df.columns

load json file...
(17, 3)


C:\Users\chaku\assess_ai\Assessment module\AUTOCOMPLETE NLP\autocompleter.py:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  column_as_df = json_normalize(df[column])


((17, 3), Index(['IsFromCustomer', 'Text', 'index'], dtype='object'))

The file contains 22K conversations between a customer and a representative.
For the purpose of this project, we are only interested in completing the threads of the representative.

In [15]:
df.tail(10)

,IsFromCustomer,Text,index
7,False,angle less for hand PLease keep your arms stra...,2
8,False,angle good for arms,3
9,False,for bicepcurl angle less for neck PLease keep ...,3
10,False,angle less for backbone,4
11,False,angle more for knee PLease bend more,4
12,False,angle less for legs,4
13,False,PLease keep your legs straight and stretch,4
14,False,angle less for hand,4
15,False,PLease keep your arms straight and stretch and...,4
16,False,angle good for arms,4


# Data Selection and Cleaning

The data is going to separate the threads from the customer and the representative, separate the sentenses based on the punctuation (we will keep the punctuation), the final text will be cleaned up with some light regex and only the sentense larger than 1 word will be kept.

Finally, since the representative has the tendency to ask the same question over and over again, the autocomplete is extremely useful by suggesting a complete sentense. In our case, we will count the number of occurence of the same sentense so we can use it as a feature later on and delete the duplicates.

In [7]:
new_df = autocompl.process_data(df)
new_df.shape, new_df.columns

select representative threads...
split sentenses on punctuation...
Text Cleaning using simple regex...
calculate nb words of sentenses...
count occurence of sentenses...
remove duplicates (keep last)...
(13, 5)


((13, 5),
 Index(['IsFromCustomer', 'Text', 'index', 'nb_words', 'Counts'], dtype='object'))

# Model and TFIDF matrix

A matrice of similarity is calculated based on the frequency of all the words in the data using tfidfvectorizer

In [8]:
model_tf, tfidf_matrice = autocompl.calc_matrice(new_df)

tfidf_matrice  (13, 178)


# Ranking Function

Finally, the autocomplete is calculating the similarity between the sentense in the data and the prefix of the sentense written by the representative. As a weight feature, we chose to reorder using the frequency of the most common similar sentense.

examples of auto completions

In [22]:
prefix = keygenerator([140,"backbone","bicepcurl"])
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)[0]

for bicepcurl angle less for backbone      


'Angle less for backbone'

Now, without any uppercase and just with the important words...

In [18]:
prefix = keygenerator([35,"knee","lunges"])
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)[0]

 good work      


'Good work continue excercise'

# Online Sources for this project

In [11]:
# https://gist.github.com/jlln/338b4b0b55bd6984f883 modified to keep punctuation
# kaggle google store competition for json read
# https://www.kaggle.com/hamishdickson/weighted-word-autocomplete-using-star-wars-dataset

In [12]:
#